In [1]:
%matplotlib inline
# standard
import sys
import os

# pandas
import pandas as pd

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# needed for project imports
sys.path.append(os.path.join(os.getcwd(), "../.."))

# project imports
from housepredictor.extractor import DictMultiExtractor


# this styling is purely my preference
# less chartjunk
sns.set_context('notebook', font_scale=1.5, rc={'line.linewidth': 2.5})
sns.set(style='ticks', palette='Set2')

In [7]:
# read the data 
raw_data = pd.read_json('../data/raw/scrape-results.json')
raw_dict = raw_data.loc[0, 'data']
raw_dict

{'AangebodenSinds': '/Date(1497823200000+0200)/',
 'AangebodenSindsTekst': '5 dagen',
 'AanmeldDatum': '/Date(1262300400000+0100)/',
 'AantalBadkamers': 2,
 'AantalKamers': 6,
 'AantalKavels': None,
 'AantalSlaapkamers': None,
 'AantalWoonlagen': '3 woonlagen',
 'Aanvaarding': 'In overleg',
 'Adres': 'Nico Jessekade 27',
 'AfgekochtDatum': '/Date(2742764400000+0100)/',
 'Afstand': 0,
 'BalkonDakterras': None,
 'BedrijfsruimteCombinatieObject': None,
 'BezichtingDagdelen': [{'Naam': 'Geen voorkeur', 'Waarde': 'Geen voorkeur'},
  {'Naam': "'s ochtends", 'Waarde': "'s ochtends"},
  {'Naam': "'s middags", 'Waarde': "'s middags"}],
 'BezichtingDagen': [{'Naam': 'Geen voorkeur', 'Waarde': 'Geen voorkeur'},
  {'Naam': 'Alleen werkdagen', 'Waarde': 'Alleen werkdagen'},
  {'Naam': 'Maandag', 'Waarde': 'Maandag'},
  {'Naam': 'Dinsdag', 'Waarde': 'Dinsdag'},
  {'Naam': 'Woensdag', 'Waarde': 'Woensdag'},
  {'Naam': 'Donderdag', 'Waarde': 'Donderdag'},
  {'Naam': 'Vrijdag', 'Waarde': 'Vrijdag'}],
 

In [4]:
KEY_NAMES = [
'AangebodenSindsTekst',
'AanmeldDatum',
'AantalKamers',
'AantalKavels',
'Aanvaarding',
'Adres',
'Afstand',
'BronCode',
'DatumOndertekeningAkte',
'GewijzigdDatum',
'GlobalId',
'HeeftOpenhuizenTopper',
'HeeftOverbruggingsgrarantie',
'HeeftTophuis',
'HeeftVeiling',
'InUnitsVanaf',
'IsVerkocht',
'IsVerkochtOfVerhuurd',
'Koopprijs',
'KoopprijsTot',
'Note',
'Oppervlakte',
'Perceeloppervlakte',
'Postcode',
'Prijs.GeenExtraKosten',
'Prijs.OriginelePrijs',
'PromoLabel.HasPromotionLabel',
'PromoLabel.PromotionType',
'PromoLabel.RibbonColor',
'PublicatieDatum',
'PublicatieStatus',
'Soort-aanbod',
'SoortAanbod',
'StartOplevering',
'WGS84_X',
'WGS84_Y',
'WoonOppervlakteTot',
'Woonoppervlakte',
'AangebodenSinds',
'AantalBadkamers',
'AantalSlaapkamers',
'AantalWoonlagen',
'AfgekochtDatum',
'BalkonDakterras',
'BijdrageVVE',
'Bijzonderheden',
'Bouwjaar',
'Bouwvorm',
'EigendomsSituatie',
'Energielabel.Definitief',
'Energielabel.Index',
'Energielabel.Label',
'Energielabel.NietBeschikbaar',
'Energielabel.NietVerplicht',
'ErfpachtBedrag',
'Garage',
'GarageIsolatie',
'GarageVoorzieningen',
'GelegenOp',
'HoofdTuinType',
'IndBasisPlaatsing',
'Inhoud',
'IsIngetrokken',
'Isolatie',
'Ligging',
'ObjectType',
'ObjectTypeMetVoorvoegsel',
'PerceelOppervlakte',
'PermanenteBewoning',
'SchuurBerging',
'SchuurBergingIsolatie',
'SchuurBergingVoorzieningen',
'ServiceKosten',
'SoortDak',
'SoortGarage',
'SoortParkeergelegenheid',
'SoortPlaatsing',
'SoortWoning',
'ToonBezichtigingMaken',
'ToonBrochureAanvraag',
'ToonMakelaarWoningaanbod',
'ToonReageren',
'TuinLigging',
'Verwarming',
'VolledigeOmschrijving',
'Voorzieningen',
'WarmWater',
'WoonOppervlakte',
'WoonOppervlakteTot',
'KoopPrijs',
]

def extract_preliminary(data):
    extraction_specs = [{'key': key} for key in KEY_NAMES]
    extractor = DictMultiExtractor(extraction_specs, sep='.')
    return pd.DataFrame(data.apply(extractor).tolist())
    


In [5]:
data = extract_preliminary(raw_data['data'])

Remove duplicated columns

This way we will know not to extract them in the first place, removing the need for this proprocessing step.

In [5]:
def duplicate_columns(df, return_dataframe = False, verbose = False):
    '''
        a function to detect and possibly remove duplicated columns for a pandas dataframe
    '''
    from pandas.core.common import array_equivalent
    # group columns by dtypes, only the columns of the same dtypes can be duplicate of each other
    groups = df.columns.to_series().groupby(df.dtypes).groups
    duplicated_columns = []

    for dtype, col_names in groups.items():
        column_values = df[col_names]
        num_columns = len(col_names)

        # find duplicated columns by checking pairs of columns, store first column name if duplicate exist 
        for i in range(num_columns):
            column_i = column_values.iloc[:,i].values
            for j in range(i + 1, num_columns):
                column_j = column_values.iloc[:,j].values
                if array_equivalent(column_i, column_j):
                    if verbose: 
                        print("column {} is a duplicate of column {}".format(col_names[i], col_names[j]))
                    duplicated_columns.append(col_names[i])
                    break
    if not return_dataframe:
        # return the column names of those duplicated exists
        return duplicated_columns
    else:
        # return a dataframe with duplicated columns dropped 
        return df.drop(labels = duplicated_columns, axis = 1)


In [6]:
duplicate_col_names = duplicate_columns(data)
deduplicated_data = duplicate_columns(data, return_dataframe=True)
data = deduplicated_data

/home/nikitautiu/anaconda2/envs/datascience3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 'pandas.core.common.array_equivalent' is deprecated and is no longer public API


Remove columns with too many missing values

Analyze what columns still have `object` as type, meaning that they are either text or should have more preprocessing done on them.

In [7]:
non_num_cols = data.dtypes[data.dtypes == object].index.tolist()
str_cols = data[non_num_cols]
print('\n'.join(non_num_cols))
str_cols.head()

AangebodenSindsTekst
AanmeldDatum
AantalWoonlagen
Aanvaarding
Adres
AfgekochtDatum
BalkonDakterras
Bijzonderheden
Bouwjaar
Bouwvorm
BronCode
EigendomsSituatie
Energielabel.Label
Garage
GarageIsolatie
GarageVoorzieningen
GelegenOp
GewijzigdDatum
HoofdTuinType
Isolatie
Ligging
ObjectType
ObjectTypeMetVoorvoegsel
PermanenteBewoning
Postcode
PublicatieDatum
SchuurBerging
SchuurBergingIsolatie
SchuurBergingVoorzieningen
Soort-aanbod
SoortDak
SoortParkeergelegenheid
SoortWoning
StartOplevering
TuinLigging
Verwarming
VolledigeOmschrijving
Voorzieningen
WarmWater


,AangebodenSindsTekst,AanmeldDatum,AantalWoonlagen,Aanvaarding,Adres,AfgekochtDatum,BalkonDakterras,Bijzonderheden,Bouwjaar,Bouwvorm,...,Soort-aanbod,SoortDak,SoortParkeergelegenheid,SoortWoning,StartOplevering,TuinLigging,Verwarming,VolledigeOmschrijving,Voorzieningen,WarmWater
0,5 dagen,/Date(1262300400000+0100)/,3 woonlagen,In overleg,Nico Jessekade 27,/Date(2742764400000+0100)/,None,,2008,bestaande bouw,...,woonhuis,plat dak bedekt met bitumineuze dakbedekking,"eengezinswoning, geschakelde woning","eengezinswoning, geschakelde woning",None,gelegen op het noordoosten,stadsverwarming,**PERFECT FAMILIEHUIS VAN CIRCA 157 M² WAAR U...,mechanische ventilatie en TV kabel,centrale voorziening
1,2 dagen,/Date(1262300400000+0100)/,2 woonlagen en een zolder,In overleg,Bombraak 31,/Date(2302210800000+0100)/,None,,1993,bestaande bouw,...,woonhuis,plat dak bedekt met bitumineuze dakbedekking,"eengezinswoning, tussenwoning","eengezinswoning, tussenwoning",None,gelegen op het zuiden,C.V.-ketel,Stel je voor: gezellig samen genieten van het ...,"alarminstallatie, buitenzonwering, jacuzzi, me...",C.V.-ketel
2,2 dagen,/Date(1262300400000+0100)/,3 woonlagen,In overleg,Raphaëlplein 39,None,balkon aanwezig,,1932,bestaande bouw,...,woonhuis,samengesteld dak bedekt met dakpannen en bitum...,"herenhuis, hoekwoning","herenhuis, hoekwoning",None,None,C.V.-ketel,"Een karatiristiek herenhuis uit de jaren 30, g...","mechanische ventilatie, rolluiken, alarminstal...",C.V.-ketel
3,2½ maand,/Date(1262300400000+0100)/,1 woonlaag,In overleg,Stadionweg 198 III,/Date(2699215200000+0200)/,balkon aanwezig,,1931,bestaande bouw,...,appartement,samengesteld dak bedekt met dakpannen en bitum...,bovenwoning (appartement),bovenwoning (appartement),None,None,blokverwarming,Zeer ruim en sfeervol 5-kamerappartement met z...,TV kabel,centrale voorziening
4,2½ maand,/Date(1262300400000+0100)/,1 woonlaag,In overleg,Ben van Meerendonkstraat 293,/Date(2796588000000+0200)/,None,,2001-2010,bestaande bouw,...,appartement,plat dak bedekt met bitumineuze dakbedekking,bovenwoning (appartement),bovenwoning (appartement),None,None,stadsverwarming en vloerverwarming geheel,"Ben van Meerendonkstraat 293, 1087 LN AMSTERDA...","mechanische ventilatie, TV kabel en lift",centrale voorziening


Analyze which columns have the most missing data and remove them. Some might have null instead of a false value, but others might just be irrelevant to the analysis. Manual selection of features.

In [10]:
data = deduplicated_data
good_cols = [col_name for col_name in data.columns if data[col_name].unique().size >= 2]
good_cols

['AangebodenSindsTekst',
 'AantalBadkamers',
 'AantalKamers',
 'AantalWoonlagen',
 'Aanvaarding',
 'Adres',
 'AfgekochtDatum',
 'BalkonDakterras',
 'BijdrageVVE',
 'Bijzonderheden',
 'Bouwjaar',
 'Bouwvorm',
 'BronCode',
 'EigendomsSituatie',
 'Energielabel.Definitief',
 'Energielabel.Index',
 'Energielabel.Label',
 'Energielabel.NietBeschikbaar',
 'Energielabel.NietVerplicht',
 'ErfpachtBedrag',
 'Garage',
 'GarageIsolatie',
 'GarageVoorzieningen',
 'GelegenOp',
 'GewijzigdDatum',
 'GlobalId',
 'HeeftVeiling',
 'HoofdTuinType',
 'Inhoud',
 'Isolatie',
 'Koopprijs',
 'Ligging',
 'ObjectType',
 'ObjectTypeMetVoorvoegsel',
 'PerceelOppervlakte',
 'Perceeloppervlakte',
 'PermanenteBewoning',
 'Postcode',
 'PromoLabel.HasPromotionLabel',
 'PromoLabel.PromotionType',
 'PromoLabel.RibbonColor',
 'PublicatieDatum',
 'SchuurBerging',
 'SchuurBergingIsolatie',
 'SchuurBergingVoorzieningen',
 'ServiceKosten',
 'Soort-aanbod',
 'SoortDak',
 'SoortParkeergelegenheid',
 'SoortPlaatsing',
 'SoortWon